In [1]:
project_name='Macbooks-comparison'

import os
import numpy as np
import plotly
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import json 
import pandas as pd

#add ../pybeme to the path
import sys
sys.path.append('../pybeme')
sys.path.append('..')

from pybeme.beme_experiment import load_experiment_results, extract_dataframe
from pybeme.reference_set import naive_pareto_front

# Upload the results.

folder_path=os.path.join(os.getcwd(), '..')

print(f"Using folder '{folder_path}' for the output results")

MCBPRO = load_experiment_results(os.path.join(folder_path, 'data',
                        'bemeopt__nsga2__anytown_rehab_f1__exp04', 'output', 'bemeopt__nsga2__anytown_rehab_f1__exp04__exp.json') )

MCBAIR = load_experiment_results(os.path.join(folder_path,'data',
                        'bemeopt__nsga2__anytown_rehab_f1__exp05', 'output','bemeopt__nsga2__anytown_rehab_f1__exp05__exp.json') )


from datetime import datetime

# times vs generation
fig = make_subplots(rows=2, cols=1, shared_xaxes=True, vertical_spacing=0.1, subplot_titles=('Time vs generations', 'Simulation speed vs generations'))
_names = ['MCBPRO', 'MCBAIR']
_colors = ['#29378A', '#808080', '#74BDA7','#02B9EA', '#F78132']
for e, exp in enumerate([MCBPRO, MCBAIR]):
    t_finals = []
    fr_all = []
    fr_all_avg = 0
    fr_all_std = 0
    for i, island in enumerate(exp['archipelago']['islands']):
        fevals = np.array([generation['fitness-evaluations'] for generation in island['generations']])
        times = np.array([datetime.strptime(generation['current-time'], "%a %b %d %H:%M:%S %Y") for generation in island['generations']])
        times = [(t-times[0]).total_seconds() for t in times]
        if i == 0:
            _showleg = True
        else:
            _showleg = False
        fig.add_trace(go.Scatter(x=fevals/100, y=[t/60 for t in times], mode='lines', line=dict(color=_colors[e], width=3), showlegend=_showleg, name=_names[e-2], legendgroup=_names[e-2]), row=1, col=1)
        fig.add_trace(go.Scatter(x=fevals[:-1]/100, y=[(fevals[i+1]-fevals[i])/((times[i+1]-times[i])) for i in range(len(times)-1)], mode='lines', line=dict(color=_colors[e], width=3), showlegend=False, name=_names[e-2], legendgroup=_names[e-2]), row=2, col=1)
        
        t_finals.append(times[-1])
        fr_all.extend([(fevals[i+1]-fevals[i])/((times[i+1]-times[i])) for i in range(len(times)-1)])
    
    print(f"Total time for the experiment on {_names[e]}: {np.mean(t_finals)/60:.2f} +- {np.std(t_finals)/60:.2f} min")
    print(f"Simulation speed for the experiment on {_names[e]}: {np.mean(fr_all):.2f} +- {np.std(fr_all):.2f} EPS/s")

fig.update_xaxes(title_text='Generation', row=2, col=1)
fig.update_yaxes(title_text='Time [min]', row=1, col=1)
fig.update_yaxes(title_text='Simulation speed [evals/s]', row=2, col=1)
fig.show()

Naive Pareto Front: [[1 2]]
Using folder '/Users/zannads/repos/BevarMejo/visual/..' for the output results
Total time for the experiment on MCBPRO: 46.95 +- 1.77 min
Simulation speed for the experiment on MCBPRO: 930.02 +- 82.95 EPS/s
Total time for the experiment on MCBAIR: 68.99 +- 0.88 min
Simulation speed for the experiment on MCBAIR: 629.59 +- 32.19 EPS/s


In [2]:
rehab_60min = load_experiment_results(os.path.join(folder_path, 'data',
                        'bemeopt__nsga2__anytown_rehab_f1__exp04', 'output', 'bemeopt__nsga2__anytown_rehab_f1__exp04__exp.json') )

rehab_30min = load_experiment_results(os.path.join(folder_path,'data',
                        'bemeopt__nsga2__anytown_rehab_f1__exp06', 'output','bemeopt__nsga2__anytown_rehab_f1__exp06__exp.json') )

rehab_15min = load_experiment_results(os.path.join(folder_path,'data',
                        'bemeopt__nsga2__anytown_rehab_f1__exp07', 'output','bemeopt__nsga2__anytown_rehab_f1__exp07__exp.json') )


# times vs generation
fig = make_subplots(rows=2, cols=1, shared_xaxes=True, vertical_spacing=0.1, subplot_titles=('Time vs generations', 'Simulation speed vs generations'))
_names = ["60 min", "30 min", "15 min"]
for e, exp in enumerate([rehab_60min, rehab_30min, rehab_15min]):
    t_finals = []
    fr_all = []
    fr_all_avg = 0
    fr_all_std = 0
    for i, island in enumerate(exp['archipelago']['islands']):
        fevals = np.array([generation['fitness-evaluations'] for generation in island['generations']])
        times = np.array([datetime.strptime(generation['current-time'], "%a %b %d %H:%M:%S %Y") for generation in island['generations']])
        times = [(t-times[0]).total_seconds() for t in times]
        if i == 0:
            _showleg = True
        else:
            _showleg = False
        fig.add_trace(go.Scatter(x=fevals/100, y=[t/60 for t in times], mode='lines', line=dict(color=_colors[e], width=3), showlegend=_showleg, name=_names[e], legendgroup=_names[e]), row=1, col=1)
        fig.add_trace(go.Scatter(x=fevals[:-1]/100, y=[(fevals[j+1]-fevals[j])/((times[j+1]-times[j])) for j in range(len(times)-1)], mode='lines', line=dict(color=_colors[e], width=3), showlegend=False, name=_names[e], legendgroup=_names[e]), row=2, col=1)
        
        t_finals.append(times[-1])
        fr_all.extend([(fevals[j+1]-fevals[j])/((times[j+1]-times[j])) for j in range(len(times)-1)])
    
    print(f"Total time for the experiment on {_names[e]}: {np.mean(t_finals)/60:.2f} +- {np.std(t_finals)/60:.2f} min")
    print(f"Simulation speed for the experiment on {_names[e]}: {np.mean(fr_all):.2f} +- {np.std(fr_all):.2f} EPS/s")

fig.update_xaxes(title_text='Generation', row=2, col=1)
fig.update_yaxes(title_text='Time [min]', row=1, col=1)
fig.update_yaxes(title_text='Simulation speed [evals/s]', row=2, col=1)
fig.show()

Total time for the experiment on 60 min: 46.95 +- 1.77 min
Simulation speed for the experiment on 60 min: 930.02 +- 82.95 EPS/s
Total time for the experiment on 30 min: 332.09 +- 1.34 min
Simulation speed for the experiment on 30 min: 130.53 +- 2.41 EPS/s
Total time for the experiment on 15 min: 664.00 +- 1.49 min
Simulation speed for the experiment on 15 min: 65.28 +- 1.09 EPS/s


In [5]:
mixed_60min = load_experiment_results(os.path.join(folder_path, 'data',
                        'bemeopt__nsga2__anytown_mixed_f1__exp03', 'output', 'bemeopt__nsga2__anytown_mixed_f1__exp03__exp.json') )

mixed_30min = load_experiment_results(os.path.join(folder_path, 'data',
                        'bemeopt__nsga2__anytown_mixed_f1__exp04', 'output','bemeopt__nsga2__anytown_mixed_f1__exp04__exp.json') )

mixed_15min = load_experiment_results(os.path.join(folder_path,'data',
                        'bemeopt__nsga2__anytown_mixed_f1__exp05', 'output','bemeopt__nsga2__anytown_mixed_f1__exp05__exp.json') )


# times vs generation
fig = make_subplots(rows=2, cols=1, shared_xaxes=True, vertical_spacing=0.1, subplot_titles=('Time vs generations', 'Simulation speed vs generations'))
_names = ["60 min", "30 min", "15 min"]
for e, exp in enumerate([mixed_60min, mixed_30min, mixed_15min]):
    t_finals = []
    fr_all = []
    fr_all_avg = 0
    fr_all_std = 0
    for i, island in enumerate(exp['archipelago']['islands']):
        fevals = np.array([generation['fitness-evaluations'] for generation in island['generations']])
        times = np.array([datetime.strptime(generation['current-time'], "%a %b %d %H:%M:%S %Y") for generation in island['generations']])
        times = [(t-times[0]).total_seconds() for t in times]
        if i == 0:
            _showleg = True
        else:
            _showleg = False
        fig.add_trace(go.Scatter(x=fevals/100, y=[t/60 for t in times], mode='lines', line=dict(color=_colors[e], width=3), showlegend=_showleg, name=_names[e], legendgroup=_names[e]), row=1, col=1)
        fig.add_trace(go.Scatter(x=fevals[:-1]/100, y=[(fevals[j+1]-fevals[j])/((times[j+1]-times[j])) for j in range(len(times)-1)], mode='lines', line=dict(color=_colors[e], width=3), showlegend=False, name=_names[e], legendgroup=_names[e]), row=2, col=1)
        
        t_finals.append(times[-1])
        fr_all.extend([(fevals[j+1]-fevals[j])/((times[j+1]-times[j])) for j in range(len(times)-1)])
    
    print(f"Total time for the experiment on {_names[e]}: {np.mean(t_finals)/60:.2f} +- {np.std(t_finals)/60:.2f} min")
    print(f"Simulation speed for the experiment on {_names[e]}: {np.mean(fr_all):.2f} +- {np.std(fr_all):.2f} EPS/s")

fig.update_xaxes(title_text='Generation', row=2, col=1)
fig.update_yaxes(title_text='Time [min]', row=1, col=1)
fig.update_yaxes(title_text='Simulation speed [evals/s]', row=2, col=1)
fig.show()

Total time for the experiment on 60 min: 350.22 +- 74.16 min
Simulation speed for the experiment on 60 min: 152.92 +- 36.53 EPS/s
Total time for the experiment on 30 min: 357.03 +- 31.84 min
Simulation speed for the experiment on 30 min: 141.27 +- 12.43 EPS/s
Total time for the experiment on 15 min: 761.29 +- 31.13 min
Simulation speed for the experiment on 15 min: 66.55 +- 7.28 EPS/s
